In [ ]:
!pip -q install --upgrade transformers accelerate bitsandbytes sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 16.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
# login("<YOUR_HF_TOKEN>")


In [ ]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # or "Qwen/Qwen2.5-0.5B-Instruct"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

use_4bit = True  # set False if you prefer full precision

dtype = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16

tokenizer = AutoTokenizer.from_pretrained(model_id)

if use_4bit:
    # Memory efficient loading for small VRAM GPUs (needs bitsandbytes)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        load_in_4bit=True
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=dtype,
        device_map="auto"
    )

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # return_full_text=False lets us see just the new tokens for some models
    return_full_text=False,
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
prompt = """Explain me whats a transformer, assuming im a 5 years old kid"""
outputs = generator(
    prompt,
    max_new_tokens=200,
    temperature=0.1
)
print(outputs[0]["generated_text"])

?

(Voiceover) A transformer is a robotic device that can transform into a variety of different forms.

(Voiceover) Imagine a world where robots could transform into anything, from a giant robot to a tiny robotic hand.

(Voiceover) But what if you could transform into something else?

(Voiceover) Imagine a world where you could transform into a transformer.

(Voiceover) A world where robots could transform into anything, and you could transform into anything.

(Voiceover) But what if you could transform into something else?

(Voiceover) Imagine a world where you could transform into a transformer.

(Voiceover) A world where robots could transform into anything, and you could transform into anything.

(Voiceover) But what if you could transform
